# XY Belt Calibration

The OT1 uses elastic bands to travel along its X (left-right) and Y (forward-backward) axis. Very slight changes in length from stretching will cause the robot to slowly lose its ability to measure distances accurately. For example, we tell the robot to move 100mm, but it seems to move about 103mm.

There is a solution! What we can do is simply tell the robot to move a certain distance, then measure and tell it how far it actually travelled. If done accurately, this will then fix the robot's internal calculations for determining "real-world" distances.

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><strong>NOTE</strong></p>
<p>Distance accuracy is different from consistency. If your robot cannot move to the same location consistently, this is a different problem and more to do with the tension of your belts. Please contact Opentrons support to help troubleshoot.</p>
</div>

## Instructions

Make sure your OT1 is connected to your computer, and the App or any other program is **not** connected to it.

Run the cell below, and it will ask you which port to connect to, and also what distances to move. After running the cell, scroll to the bottom of this screen to follow along with the print-outs.

### Video Tutorial on Calibrating XY belts

Follow along with [this video tutorial](https://www.youtube.com/watch?v=Y2Xd7FFzQLk). It will show you how to run this notebook and measure your robot's XY belts.

### Tools Required

Some [painter's tape](https://www.amazon.com/ScotchBlue-Painters-Multi-Use-1-41-Inch-60-Yard/dp/B00004Z4DI/ref=sr_1_4?s=hi&ie=UTF8&qid=1486139289&sr=1-4&keywords=painters+tape), a pen, and an accurate measuring tool like a ruler.

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><strong>NOTE</strong></p>
<p>Be very careful when changing the XY belt calibrations, as innacurate setting could make the robot move uncontrollably. If things go wrong and you wish to start fresh, you can always <a href="http://docs.opentrons.com/updating_firmware.html">reload the firmware</a>.</p>
</div>

In [ ]:
from opentrons import robot


def connect_to_robot():
    """
    Search for ports, connect to a robot, and home
    """
    print('Enter the number of the port to connect to:\n')
    count = 0
    ports = robot.get_serial_ports_list()
    if not ports:
        raise RuntimeError(
            'No serial ports found. Please make sure your robot\'s USB cable is connected')
    for i, portname in enumerate(ports):
        print('  {0})   {1}\n'.format(count, portname))
    val = input('Enter number next to desired port (0, 1, etc): '.format(1, count))
    try:
        val = int(val)
    except:
        print('Bad input: {}'.format(val))
        sys.exit()
    try:
        robot.connect(robot.get_serial_ports_list()[val])
    except RuntimeWarning:
        robot.connect(robot.get_serial_ports_list()[val])
    print('Connected, homing...')
    robot.home()
    robot.home('xy')


def request_distance_from_user():
    """
    Get the dimensions of this robot, and ask the user for the testing distance
    """
    dimensions = robot._driver.get_dimensions() - (50, 50, 0)
    x_dist = int(input('Enter distanct for X to travel (max {}):'.format(dimensions[0])))
    y_dist = int(input('Enter distanct for Y to travel (max {}):'.format(dimensions[1])))
    x_dist = min(x_dist, dimensions[0])
    y_dist = min(y_dist, dimensions[1])
    return (x_dist, y_dist)

    
def measure_belts(x_dist, y_dist):
    """
    Move the robot, allow user to mark positions, ask user to measure and submit measurements
    """
    print('Moving the robot {0}mm on the X, and {1}mm on the Y'.format(x_dist, y_dist))
    robot.move_head(x=x_dist, y=y_dist * -1, mode='relative')  # Y axis is flipped

    input('mark X position on the robot, then press ENTER key')
    input('mark Y position on the robot, then press ENTER key')

    robot.home('xy')
    robot.home('xy')

    actual_travel_x = float(input('enter the measured X distance: '))
    actual_travel_y = float(input('enter the measured Y distance: '))
    robot._driver.calibrate_steps_per_mm('x', x_dist, actual_travel_x)
    robot._driver.calibrate_steps_per_mm('y', y_dist, actual_travel_y)


connect_to_robot()
x, y = request_distance_from_user()
measure_belts(x, y)

print('New values saved. Restarting the robot to allow changes to take effect')
print('Please wait 5-10 seconds before reconnecting to the robot')

robot._driver.reset()
robot.disconnect()

### follow the print-outs you'll see above ^^^